In [1]:
# Imports
import numpy as np
import pandas as pd

from scipy.spatial import distance

# from sklearn.preprocessing import MinMaxScaler
# pip3 install pandas ipython jupyter

## Preprocesado csv de las encuestas

In [2]:
COMPANIES_ENCUESTAS_CSV = './encuestas/Web - Ironhack Matchmaking Company Form.csv'
companies_enc = pd.read_csv(COMPANIES_ENCUESTAS_CSV, sep=',', index_col=0, lineterminator='\n')
# display(companies_enc.columns)
DIC = {
    "What is the name of your company?": "name", 
    'Does your company have specific language requirements for this position? [English]': 'english', 
    'Does your company have specific language requirements for this position? [Spanish]': 'spanish', 
    'Does your company have specific language requirements for this position? [Portuguese]': 'portuguese', 
    'Does your company have specific language requirements for this position? [French]': 'french', 
    'Does your company have specific language requirements for this position? [Dutch]': 'dutch', 
    'Does your company have specific language requirements for this position? [Catalan]': 'catalan', 
    'Where will this position be located?': 'location', 
    'Regarding remote working, how many days a week could the employee work off site?': 'offsite', 
    'On a scale from 1 to 10, how much time will the employee be spending working on back end and front end?': 'position', 
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Java]': 'java', 
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [C# / ASP.net]': 'caspnet',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Python]': 'python',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [PHP]': 'php',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [SQL]': 'sql',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Angular (Typescript generation)]': 'angular',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Vue]': 'vue',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Firebase]': 'firebase',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [AWS]': 'aws',
    'Aside from the ones covered in our curriculum (MERN Stack), are there additional specific hard-skills you would value in an employee? [Docker / Kubernetes]': 'dockerkubernetes',
    'From this list, please rank these 3 soft skills in order of importance for this position [Motivation and ability to overcome problems: employee has a passionate, can-do attitude and proactively looks for solutions to every problem they encounter]': 'motivation',
    'From this list, please rank these 3 soft skills in order of importance for this position [Coachability: Employee is receptive and actively listens and acts on feedback received]': 'coachability',
    'From this list, please rank these 3 soft skills in order of importance for this position [Teamwork: Employee is able to receive and provide value in a team environment]': 'teamwork'
    }
companies_enc.rename(index=str, columns=DIC, inplace=True)
companies_enc.index = companies_enc.name
companies_enc.drop(columns=['name'], axis=1, inplace=True)
# display(companies_enc.columns)
companies_enc.head()

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,Must be able to read documentation and communi...,Can work in full capacity in this language,Can work in full capacity in this language,NaN,NaN,NaN,In the country where the campus is located,1-2 Days a week,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
Company B,Must be able to read documentation and communi...,Can work in full capacity in this language,Can work in full capacity in this language,NaN,NaN,NaN,In a city in another country,100% office-based job. We're not keen on remot...,8,NaN,...,NaN,NaN,Must have: Must be able to work with the techn...,NaN,NaN,NaN,NaN,2,3,1
Company C,Can work in full capacity in this language,Can work in full capacity in this language,Can work in full capacity in this language,NaN,Must be able to read documentation and have an...,NaN,In a city in another country,1-2 Days a week,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3


In [3]:
def translate_language(x): 
    RESPONSES_LANGUAGE = [
        'Must be able to read basic documentation', 
        'Must be able to read documentation and have an informal conversation', 
        'Must be able to read documentation and communicate in a meeting', 
        'Can work in full capacity in this language'
    ]
    # +1 para diferenciar de la no obligatoria
    return 0 if x not in RESPONSES_LANGUAGE else RESPONSES_LANGUAGE.index(x) + 1 


LANGUAGES = ['english', 'spanish', 'portuguese', 'french', 'dutch', 'catalan']
for language in LANGUAGES: 
    companies_enc[language] = companies_enc[language].apply(translate_language)
companies_enc.head()

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,3,4,4,0,0,0,In the country where the campus is located,1-2 Days a week,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
Company B,3,4,4,0,0,0,In a city in another country,100% office-based job. We're not keen on remot...,8,NaN,...,NaN,NaN,Must have: Must be able to work with the techn...,NaN,NaN,NaN,NaN,2,3,1
Company C,4,4,4,0,2,0,In a city in another country,1-2 Days a week,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3


In [4]:
def translate_location(x): 
    RESPONSES_LOCATION = [
        'In the city where the campus is located', 
        'In the country where the campus is located', 
        'In a city in another country'
    ]
    return 0 if x not in RESPONSES_LOCATION else RESPONSES_LOCATION.index(x) + 1

companies_enc['location'] = companies_enc['location'].apply(translate_location)
companies_enc.head()

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,3,4,4,0,0,0,2,1-2 Days a week,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
Company B,3,4,4,0,0,0,3,100% office-based job. We're not keen on remot...,8,NaN,...,NaN,NaN,Must have: Must be able to work with the techn...,NaN,NaN,NaN,NaN,2,3,1
Company C,4,4,4,0,2,0,3,1-2 Days a week,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3


In [5]:
def translate_offsite(x): 
    RESPONSES_OFFSITE = [
        "100% office-based job. We're not keen on remote working.", 
        '1-2 Days a week', 
        "3+ Days. It's up to the employee", 
        "100% remote-based job. No physical office."
    ]
    return 0 if x not in RESPONSES_OFFSITE else RESPONSES_OFFSITE.index(x) + 1

companies_enc['offsite'] = companies_enc['offsite'].apply(translate_offsite)
companies_enc.head()  

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,3,4,4,0,0,0,2,2,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
Company B,3,4,4,0,0,0,3,1,8,NaN,...,NaN,NaN,Must have: Must be able to work with the techn...,NaN,NaN,NaN,NaN,2,3,1
Company C,4,4,4,0,2,0,3,2,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3


In [6]:
def translate_position(x): 
    return int(x)

companies_enc['position'] = companies_enc['position'].apply(translate_position)
companies_enc.head() 

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,3,4,4,0,0,0,2,2,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3
Company B,3,4,4,0,0,0,3,1,8,NaN,...,NaN,NaN,Must have: Must be able to work with the techn...,NaN,NaN,NaN,NaN,2,3,1
Company C,4,4,4,0,2,0,3,2,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3


In [9]:
def translate_hardskills(x): 
    RESPONSES_HARDSKILLS = [
        'Nice to have: They should have a basic knowledge', 
        'Must have: Must be able to work with the technology on a regular basis'
    ]
    # +1 para diferenciar de la no obligatoria
    return 0 if x not in RESPONSES_HARDSKILLS else RESPONSES_HARDSKILLS.index(x) + 1 


HARDSKILLS = ['java', 'caspnet', 'python', 'php', 'sql', 'angular', 
              'vue', 'firebase', 'aws', 'dockerkubernetes']

for hardskill in HARDSKILLS: 
    companies_enc[hardskill] = companies_enc[hardskill].apply(translate_hardskills)
companies_enc.head()  
    

,english,spanish,portuguese,french,dutch,catalan,location,offsite,position,java,...,php,sql,angular,vue,firebase,aws,dockerkubernetes,motivation,coachability,teamwork
name,,,,,,,,,,,,,,,,,,,,,
Company A,3,4,4,0,0,0,2,2,4,0,...,0,0,0,0,0,0,0,1,2,3
Company B,3,4,4,0,0,0,3,1,8,0,...,0,0,2,0,0,0,0,2,3,1
Company C,4,4,4,0,2,0,3,2,3,0,...,0,0,0,0,0,0,0,1,2,3


In [8]:
companies_enc.columns

Index(['english', 'spanish', 'portuguese', 'french', 'dutch', 'catalan',
       'location', 'offsite', 'position', 'java', 'caspnet', 'python', 'php',
       'sql', 'angular', 'vue', 'firebase', 'aws', 'dockerkubernetes',
       'motivation', 'coachability', 'teamwork'],
      dtype='object')

## Tratado de los CSVs

In [ ]:
# Alumnos

STUDENTS_CSV = './input/students.csv'
COMPANIES_CSV = './input/companies.csv'

def csv2dataframe(path): 
    return pd.read_csv(path, sep=',', index_col=0, lineterminator='\n')

def adquisition(): 
    students = csv2dataframe(STUDENTS_CSV)
    companies = csv2dataframe(COMPANIES_CSV)
    return students, companies

students, companies = adquisition()

display(students), display(companies)

In [ ]:
# normalizacion. Por columnas en alumnos o empresas hay un 0.0 y un 1.0

def normalize_2dfs(students, companies): 
    
    mins = [min(s, c) for s, c in zip(students.min(), companies.min())]
    students = students-mins
    companies = companies-mins
    
    maxs = [max(s, c) for s, c in zip(students.max(), companies.max())]
    students = students/maxs
    companies = companies/maxs
    
    return students, companies

students_norm, companies_norm = normalize_2dfs(students, companies)
# display(students_norm), display(companies_norm)

In [ ]:
# Matching

# PESOS = (alumnos.columns)

P = [2,1,1,1,3,1,1,1,1,5]
W = {col: p for col, p in zip(students_norm.columns, P)}


def calc_dist(s_array, c_array, weights_val): 
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine
    return calc_match(distance.euclidean(s_array, c_array, weights_val))

def calc_match(distance): 
    return 1/(1+distance)

def match(student, company, weights=W): 
    return calc_dist(student, company, list(weights.values()))

In [ ]:
# match(students_norm.loc['alumno1'], companies_norm.loc['empresa1'])
W

# Emparejamiento

En cada ronda se crea una pareja alumno-empresa empezando por el mejor %. Luego se pasa al siguente alumno. Si la empresa que hacía mejor match es la del caso anterior, pasa a su segundo mejor match, etc. 

En cada ronda se cambia el estudiante que elige primero, entre los que no han ido primeros

In [ ]:
# !jupyter kernelspec list
# import sys
# sys.path

In [ ]:
# create dataframe de tamañ0 SxC

matching = pd.DataFrame(np.zeros((len(students), len(companies))) , columns=companies.index, index=students.index)
matching.head()

In [ ]:
matching.index, matching.columns

In [ ]:
# matching.loc['alumno1', 'empresa3']

def calculate_match(matching, df1, df2): 
    # matching = pd.DataFrame(np.zeros((len(students), len(companies))) , columns=companies.index, index=students.index)

    for a in matching.index: 
        for c in matching.columns:
            # print(a, c)
            matching.loc[a, c] = match(df1.loc[a], df2.loc[c])
    return matching

matching = calculate_match(matching, students_norm, companies_norm)

# matching es una matriz de elementos entre [0, 1]
# matching.min().min(), matching.max().max()

matching

In [ ]:
# create csv de pruebas

df_pruebas = matching.copy()     
df1 = df_pruebas.iloc[:, :15].copy() # 30x15
df2 = df_pruebas.head(15).copy()     # 15x30
df3 = df_pruebas.copy()              # 30x30

df1.shape, df2.shape, df3.shape

# Grafos

In [ ]:
import networkx as nx
import matplotlib

# https://networkx.github.io/documentation/networkx-1.10/reference/introduction.html

def create_graph(df): 
    G_res=nx.Graph()
    for a in df.index: 
        for e in df.columns: 
            G_res.add_edge(a,e,weight=df.loc[a][e])
    return G_res

def plot_bipartite_graph(G): 
    X, Y = df1.index, df1.columns
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
    nx.draw(G, pos=pos)

    '''
def get_best_match2(G, node): 
    # sort por peso DESCENCENTE
    for edge in sorted(G.edges(data=True), key=lambda x: - x[2]['weight']):
        if node in edge: 
            return edge
'''    
# sort por peso DESCENCENTE

def get_best_match(G, node, used): 
    # sort por peso DESCENCENTE
    for edge in sorted(G.edges(data=True), key=lambda x: - x[2]['weight']):
        if node in edge and all(u not in edge for u in used): 
            return edge
 
    


# más alumnos que empresas favoreciendo empresas

In [ ]:
# node1, node2, w

G = create_graph(df1)
students = df1.index
companies = df1.columns

# plot_bipartite_graph(G)

# node1, node2, w = get_best_match(G, 'empresa3', ['alumno3'])
# node1, node2, w


RONDAS = 10
MESAS = min(len(students), len(companies))
MESAS

rondas = []

for ronda in range(RONDAS): 
    aux = []
    used = []
    for company in companies: 
        node1, node2, w = get_best_match(G, company, used)
        aux.append((node1, node2, w))
        G.remove_edge(node1, node2)
        if node1 == company: 
            used.append(node2)
        else: 
            used.append(node1)
    rondas.append(aux)
    
rondas


# favorecemos a los estudiantes

In [ ]:
# node1, node2, w

G = create_graph(df1)

students = df1.index
companies = df1.columns

# plot_bipartite_graph(G)

# node1, node2, w = get_best_match(G, 'empresa3', ['alumno3'])
# node1, node2, w


RONDAS = 10
MESAS = min(len(students), len(companies))
MESAS

rondas = []
i = 0

for ronda in range(RONDAS): 
    aux = []
    used = []
    # for company in companies: 
    while True: 
        node1, node2, w = get_best_match(G, students[i%len(students)], used)
        aux.append((node1, node2, w))
        G.remove_edge(node1, node2)
        if node1 == students[i%len(students)]: 
            used.append(node2)
        else: 
            used.append(node1)
        i += 1
        if i == len(students): 
            break
    rondas.append(aux)
    
rondas
    


In [ ]:
i = 0
students[i%len(students)]